# Hello Node Kubernetes

**Learning Objectives**
 * Create a Node.js server
 * Create a Docker container image
 * Create a container cluster and a Kubernetes pod
 * Scale up your services

## Overview

The goal of this hands-on lab is for you to turn code that you have developed into a replicated application running on Kubernetes, which is running on Kubernetes Engine. For this lab the code will be a simple Hello World node.js app.

Here's a diagram of the various parts in play in this lab, to help you understand how the pieces fit together with one another. Use this as a reference as you progress through the lab; it should all make sense by the time you get to the end (but feel free to ignore this for now).

<img src='../assets/k8s_hellonode_overview.png' width="500"/>

## Create a Node.js server

The file `./src/server.js` contains a simple Node.js server. Use `cat` to examine the contents of that file.

In [1]:
!cat ./src/server.js

var http = require('http');
var handleRequest = function(request, response) {
  response.writeHead(200);
  response.end("Hello World!\n");
}
var www = http.createServer(handleRequest);
www.listen(8000);


Start the server by running `node server.js` in the cell below. Open a terminal and type
```bash
curl http://localhost:8000
```
to see what the server outputs. 

In [3]:
!node ./src/server.js

^C


In [4]:
!curl http://localhost:8000

curl: (7) Failed to connect to localhost port 8000: Connection refused


You should see the output `"Hello World!"`. Once you've verfied this, interupt the above running cell by hitting the stop button.

## Create and build a Docker image

Now we will create a docker image called `hello_node.docker` that will do the following:

 1. Start from the node image found on the Docker hub by inhereting from `node:6.9.2`
 2. Expose port 8000
 3. Copy the `./src/server.js` file to the image
 4. Start the node server as we previously did manually using `CMD`

**Exercise**

Edit the file called called `hello_node.docker` in the `dockerfiles` folder to complete all the TODOs.

Next, build the image in your project using `docker build`.

In [5]:
import os

PROJECT_ID = "qwiklabs-gcp-01-9a9d18213c32"  # REPLACE WITH YOUR PROJECT NAME
os.environ["PROJECT_ID"] = PROJECT_ID

**Exercise**

Use `docker build` to build the image from dockerfile you created above. Tag the image as `gcr.io/[project-id]/hello-node:v1`.

In [11]:
!docker build -t test:test . -f dockerfiles/hello_node.docker

Sending build context to Docker daemon  134.7kB
Step 1/5 : FROM node:stretch-slim
 ---> 52de4a21b501
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> 20b337a04125
Step 3/5 : COPY ./src /app
 ---> Using cache
 ---> 28755aaec5ee
Step 4/5 : EXPOSE 8000
 ---> Using cache
 ---> 19ac46956ef1
Step 5/5 : CMD [ "node", "server.js" ]
 ---> Using cache
 ---> e9195835a3c5
Successfully built e9195835a3c5
Successfully tagged test:test


In [14]:
!docker tag test:test gcr.io/qwiklabs-gcp-01-9a9d18213c32/hello-node:v1

It'll take some time to download and extract everything, but you can see the progress bars as the image builds. Once complete, test the image locally by running a Docker container as a daemon on port 8000 from your newly-created container image.

**Exercise**

Run the container using `docker run` on the port 8000.

In [20]:
!docker images

REPOSITORY                                       TAG            IMAGE ID       CREATED         SIZE
test                                             test           e9195835a3c5   3 minutes ago   216MB
gcr.io/qwiklabs-gcp-01-9a9d18213c32/hello-node   v1             e9195835a3c5   3 minutes ago   216MB
gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app     0.2            853f4b2bbbff   22 hours ago    216MB
<none>                                           <none>         232484aa70da   22 hours ago    216MB
<none>                                           <none>         b0453864e62e   22 hours ago    216MB
node                                             stretch-slim   52de4a21b501   5 days ago      216MB
node                                             latest         ba17ecfd099c   5 days ago      991MB
gcr.io/inverting-proxy/agent                     <none>         fe507176d0e6   13 months ago   1.73GB


In [21]:
!docker ps

CONTAINER ID   IMAGE                                               COMMAND                  CREATED          STATUS          PORTS                            NAMES
cc1003c9e2b4   gcr.io/qwiklabs-gcp-01-9a9d18213c32/hello-node:v1   "docker-entrypoint.s…"   21 seconds ago   Up 20 seconds   8000/tcp                         fervent_nobel
1abfbfcae857   gcr.io/qwiklabs-gcp-01-9a9d18213c32/node-app:0.2    "docker-entrypoint.s…"   21 hours ago     Up 21 hours     4000/tcp, 0.0.0.0:4000->80/tcp   pensive_goldwasser
bd408fa7ce9b   853f4b2bbbff                                        "docker-entrypoint.s…"   22 hours ago     Up 22 hours     0.0.0.0:80->80/tcp, 4000/tcp     trusting_almeida
ddc3d7eb3a49   gcr.io/inverting-proxy/agent                        "/bin/sh -c '/opt/bi…"   22 hours ago     Up 22 hours                                      proxy-agent


In [18]:
!docker run -d gcr.io/qwiklabs-gcp-01-9a9d18213c32/hello-node:v1 

cc1003c9e2b44110738c8fa052ff428191de41caf37ea0cd86f0938e9d254dc3


Your output should look something like this:
```bash
b16e5ccb74dc39b0b43a5b20df1c22ff8b41f64a43aef15e12cc9ac3b3f47cfd
```

Right now, since you used the `--d` flag, the container process is running in the background. You can verify it's running using `curl` as before.

In [23]:
!curl http://localhost:8000

curl: (7) Failed to connect to localhost port 8000: Connection refused


Now, stop running the container. Get the container id using `docker ps` and then terminate using `docker stop`

In [ ]:
!docker ps

In [24]:
# your container id will be different
!docker stop cc1003c9e2b4

cc1003c9e2b4


Now that the image is working as intended, push it to the Google Container Registry, a private repository for your Docker images, accessible from your Google Cloud projects. First, configure docker uisng your local config file. The initial push may take a few minutes to complete. You'll see the progress bars as it builds.

In [25]:
!gcloud auth configure-docker


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


**Exercise**

Push the `hello-node:v1` image to your Cloud gcr.

In [26]:
!docker push gcr.io/qwiklabs-gcp-01-9a9d18213c32/hello-node:v1 

The push refers to repository [gcr.io/qwiklabs-gcp-01-9a9d18213c32/hello-node]

4f2199b5: Preparing 
2ce66471: Preparing 
25f798a2: Preparing 
b3c2e315: Preparing 
6f306d7d: Preparing 
d82b89f1: Preparing 
dd17aa8c: Layer already exists v1: digest: sha256:bf942c365a4de57060ed424efb3625fdab58543db883894d9622572b5ee20627 size: 1781


The container image will be listed in your Console. Select `Navigation` > `Container Registry`.

## Create a cluster on GKE

Now you're ready to create your Kubernetes Engine cluster. A cluster consists of a Kubernetes master API server hosted by Google and a set of worker nodes. The worker nodes are Compute Engine virtual machines.

Create a cluster with two n1-standard-1 nodes (this will take a few minutes to complete). You can safely ignore warnings that come up when the cluster builds.

**Note**: You can also create this cluster through the Console by opening the Navigation menu and selecting `Kubernetes Engine` > `Kubernetes clusters` > `Create cluster`.

**Exercise** 

Create a GKE cluster in your project using `gcloud container clusters create`. Your cluster should
 1. be named `hello-world`
 2. have two nodes
 3. consist of `n1-standard-1` machines

In [28]:
import os

CLUSTER_NAME = "asl-cluster"
ZONE = "us-central1-a"

os.environ["CLUSTER_NAME"] = CLUSTER_NAME
os.environ["ZONE"] = ZONE

In [29]:
!gcloud container clusters create $CLUSTER_NAME --zone us-central1-a

Default change: VPC-native is the default mode during cluster creation for versions greater than 1.21.0-gke.1500. To create advanced routes based clusters, please pass the `--no-enable-ip-alias` flag
Note: Your Pod address range (`--cluster-ipv4-cidr`) can accommodate at most 1008 node(s).
Creating cluster asl-cluster in us-central1-a... Cluster is being configured...
⠛                                                                              
Creating cluster asl-cluster in us-central1-a... Cluster is being health-checke
d (master is healthy)...done.                                                  
Created [https://container.googleapis.com/v1/projects/qwiklabs-gcp-01-9a9d18213c32/zones/us-central1-a/clusters/asl-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1-a/asl-cluster?project=qwiklabs-gcp-01-9a9d18213c32
kubeconfig entry generated for asl-cluster.
NAME         LOCATION       MASTER_VERSION   MA

## Create a pod

A Kubernetes pod is a group of containers tied together for administration and networking purposes. It can contain single or multiple containers. Here you'll use one container built with your `Node.js` image stored in your private container registry. It will serve content on port 8000.

**Exercise**

Create a pod with the `kubectl create` command. You should 
 1. name your deployment `hello-node`
 2. be based on the image you created above called `gcr.io/[project-id]/hello-node:v1

In [31]:
!gcloud container clusters get-credentials $CLUSTER_NAME --zone $ZONE

Fetching cluster endpoint and auth data.
kubeconfig entry generated for asl-cluster.


In [52]:
!kubectl create deployment hello-node --image gcr.io/qwiklabs-gcp-01-9a9d18213c32/hello-node:v1

deployment.apps/hello-node created


In [54]:
!kubectl expose deployment hello-node --type=LoadBalancer --name=hello-node1 --port=8000

service/hello-node1 exposed


As you can see, you've created a deployment object. Deployments are the recommended way to create and scale pods. Here, a new deployment manages a single pod replica running the `hello-node:v1` image.

View the deployment using `kubectl get`

In [50]:
!kubectl get deployments


NAME           READY   UP-TO-DATE   AVAILABLE   AGE
hello-node     1/1     1            1           4m39s
hello-server   1/1     1            1           5m18s
deployment.apps "hello-node" deleted


Similarly, view the pods created by the deployment by also using `kubectl get`

In [58]:
!kubectl get services

NAME          TYPE           CLUSTER-IP    EXTERNAL-IP       PORT(S)          AGE
hello-node    LoadBalancer   10.88.6.196   34.72.12.126      8001:31460/TCP   5m16s
hello-node1   LoadBalancer   10.88.3.124   <pending>         8000:31464/TCP   30s
kubernetes    ClusterIP      10.88.0.1     <none>            443/TCP          8m9s
my-service    LoadBalancer   10.88.13.28   104.197.134.202   8000:32251/TCP   5m33s


Here are some other good kubectl commands you should know. They won't change the state of the cluster. Others can be found [here](https://kubernetes.io/docs/reference/kubectl/overview/).
 * `kubectl cluster-info`
 * `kubectl config view`
 * `kubectl get events`
 * `kubectl logs <pod-name>`

## Allow external traffic

By default, the pod is only accessible by its internal IP within the cluster. In order to make the hello-node container accessible from outside the Kubernetes virtual network, you have to expose the pod as a Kubernetes service.

You can expose the pod to the public internet with the `kubectl expose` command. The `--type="LoadBalancer"` flag is required for the creation of an externally accessible IP. This flag specifies that are using the load-balancer provided by the underlying infrastructure (in this case the Compute Engine load balancer). Note that you expose the deployment, and not the pod, directly. This will cause the resulting service to load balance traffic across all pods managed by the deployment (in this case only 1 pod, but you will add more replicas later).

**Exercise**

Expose the pod using `kubectl expose` and using the default load-balancer. 

In [ ]:
%%bash
TODO: Your code goes here

The Kubernetes master creates the load balancer and related Compute Engine forwarding rules, target pools, and firewall rules to make the service fully accessible from outside of Google Cloud.

To find the publicly-accessible IP address of the service, request kubectl to list all the cluster services.

In [ ]:
!kubectl get services

There are 2 IP addresses listed for your `hello-node` service, both serving port 8000. The `CLUSTER-IP` is the internal IP that is only visible inside your cloud virtual network; the `EXTERNAL-IP` is the external load-balanced IP.

You should now be able to reach the service by calling `curl http://<EXTERNAL_IP>:8000`.

In [60]:
!curl http://10.88.3.124:8000

curl: (7) Failed to connect to 10.88.3.124 port 8000: Connection timed out


At this point you've gained several features from moving to containers and Kubernetes - you do not need to specify on which host to run your workload and you also benefit from service monitoring and restart. Now see what else can be gained from your new Kubernetes infrastructure.

## Scale up your service
One of the powerful features offered by Kubernetes is how easy it is to scale your application. Suppose you suddenly need more capacity. You can tell the replication controller to manage a new number of replicas for your pod: 
```bash
kubectl scale deployment hello-node --replicas=4
```

Scale your `hello-node` application to have 6 replicas. Then use `kubectl get` to request a description of the updated deployment and list all the pods:

**Exercise**

Use the `kubectl scale deployment` command to scale up the `hello-node` service to six machines.

In [ ]:
%%bash
TODO: Your code goes here

In [ ]:
!kubectl get deployment

In [ ]:
!kubectl get pods

A declarative approach is being used here. Rather than starting or stopping new instances, you declare how many instances should be running at all times. Kubernetes reconciliation loops makes sure that reality matches what you requested and takes action if needed.

Here's a diagram summarizing the state of your Kubernetes cluster:

<img src='../assets/k8s_cluster.png' width="500">

## Roll out an upgrade to your service
At some point the application that you've deployed to production will require bug fixes or additional features. Kubernetes helps you deploy a new version to production without impacting your users.

First, modify the application by opening `server.js` so that the response is 
```bash
response.end("Hello Kubernetes World!");
```

Now you can build and publish a new container image to the registry with an incremented tag (`v2` in this case).

**Note**: Building and pushing this updated image should be quicker since caching is being taken advantage of.

**Exercise**

Build and push the updated image using the `hello_node.docker` file. Tag the updated image as `gcr.io/[project-id]/hello-node:v2`.

In [ ]:
%%bash
TODO: Your code goes here
TODO: Your code goes here

Kubernetes will smoothly update your replication controller to the new version of the application. In order to change the image label for your running container, you will edit the existing `hello-node` deployment and change the image from `gcr.io/PROJECT_ID/hello-node:v1` to `gcr.io/PROJECT_ID/hello-node:v2`.

To do this, use the `kubectl edit` command. It opens a text editor displaying the full deployment yaml configuration. It isn't necessary to understand the full yaml config right now, just understand that by updating the `spec.template.spec.containers.image` field in the config you are telling the deployment to update the pods with the new image.

Open a terminal and run the following command:

```bash 
kubectl edit deployment hello-node
```

Look for `Spec` > `containers` > `image` and change the version number to `v2`. This is the output you should see:

```bash
deployment.extensions/hello-node edited
```

New pods will be created with the new image and the old pods will be deleted. Run `kubectl get deployments` to confirm. 

**Note**: You may need to rerun the above command as it provisions machines.

In [ ]:
!kubectl get deployments

While this is happening, the users of your services shouldn't see any interruption. After a little while they'll start accessing the new version of your application. You can find more details on rolling updates in this documentation.

Hopefully with these deployment, scaling, and updated features, once you've set up your Kubernetes Engine cluster, you'll agree that Kubernetes will help you focus on the application rather than the infrastructure.

## Cleanup

Delete the cluster using `gcloud` to free up those resources. Use the `--quiet` flag if you are executing this in a notebook. Deleting the cluster can take a few minutes. 

In [ ]:
!gcloud container clusters --quiet delete hello-world

Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.